<a href="https://colab.research.google.com/github/nukano0522/pytorch/blob/master/torch_bert_ner_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 8.1 MB/s 
     |████████████████████████████████| 486 kB 51.0 MB/s 
     |████████████████████████████████| 13.4 MB 50.3 MB/s 
     |████████████████████████████████| 120 kB 58.6 MB/s 
     |████████████████████████████████| 6.6 MB 38.8 MB/s 
     |████████████████████████████████| 880 kB 47.0 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=82b8c6966677853005d2dc7aea13864604de695d6e690c32ac666cffc3d30ab6
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=b5153c53009b48e523386d8dbaaa04279667cabbedb602e3cb5b99e569d4eb52
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built ipadic 

In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import itertools
import random
import json
from tqdm import tqdm
import numpy as np
import unicodedata

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification, BertModel, AdamW

# 日本語学習済みモデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [4]:
class NER_tokenizer(BertJapaneseTokenizer):
       
    def encode_plus_tagged(self, text, entities, max_length):
        """
        文章とそれに含まれる固有表現が与えられた時に、
        符号化とラベル列の作成を行う。
        """
        # 固有表現の前後でtextを分割し、それぞれのラベルをつけておく。
        entities = sorted(entities, key=lambda x: x['span'][0])
        splitted = [] # 分割後の文字列を追加していく
        position = 0
        for entity in entities:
            start = entity['span'][0]
            end = entity['span'][1]
            label = entity['type_id']
            # 固有表現ではないものには0のラベルを付与
            splitted.append({'text':text[position:start], 'label':0}) 
            # 固有表現には、固有表現のタイプに対応するIDをラベルとして付与
            splitted.append({'text':text[start:end], 'label':label}) 
            position = end
        splitted.append({'text': text[position:], 'label':0})
        splitted = [ s for s in splitted if s['text'] ] # 長さ0の文字列は除く

        # 分割されたそれぞれの文字列をトークン化し、ラベルをつける。
        tokens = [] # トークンを追加していく
        labels = [] # トークンのラベルを追加していく
        for text_splitted in splitted:
            text = text_splitted['text']
            label = text_splitted['label']
            tokens_splitted = self.tokenize(text)
            labels_splitted = [label] * len(tokens_splitted)
            tokens.extend(tokens_splitted)
            labels.extend(labels_splitted)

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = self.convert_tokens_to_ids(tokens)
        encoding = self.prepare_for_model(
            input_ids, 
            max_length=max_length, 
            padding='max_length', 
            truncation=True
        ) # input_idsをencodingに変換
        # 特殊トークン[CLS]、[SEP]のラベルを0にする。
        labels = [0] + labels[:max_length-2] + [0] 
        # 特殊トークン[PAD]のラベルを0にする。
        labels = labels + [0]*( max_length - len(labels) ) 
        encoding['labels'] = labels

        return encoding

    def encode_plus_untagged(
        self, text, max_length=None, return_tensors=None
    ):
        """
        文章をトークン化し、それぞれのトークンの文章中の位置も特定しておく。
        """
        # 文章のトークン化を行い、
        # それぞれのトークンと文章中の文字列を対応づける。
        tokens = [] # トークンを追加していく。
        tokens_original = [] # トークンに対応する文章中の文字列を追加していく。
        words = self.word_tokenizer.tokenize(text) # MeCabで単語に分割
        for word in words:
            # 単語をサブワードに分割
            tokens_word = self.subword_tokenizer.tokenize(word) 
            tokens.extend(tokens_word)
            if tokens_word[0] == '[UNK]': # 未知語への対応
                tokens_original.append(word)
            else:
                tokens_original.extend([
                    token.replace('##','') for token in tokens_word
                ])

        # 各トークンの文章中での位置を調べる。（空白の位置を考慮する）
        position = 0
        spans = [] # トークンの位置を追加していく。
        for token in tokens_original:
            l = len(token)
            while 1:
                if token != text[position:position+l]:
                    position += 1
                else:
                    spans.append([position, position+l])
                    position += l
                    break

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = self.convert_tokens_to_ids(tokens) 
        encoding = self.prepare_for_model(
            input_ids, 
            max_length=max_length, 
            padding='max_length' if max_length else False, 
            truncation=True if max_length else False
        )
        sequence_length = len(encoding['input_ids'])
        # 特殊トークン[CLS]に対するダミーのspanを追加。
        spans = [[-1, -1]] + spans[:sequence_length-2] 
        # 特殊トークン[SEP]、[PAD]に対するダミーのspanを追加。
        spans = spans + [[-1, -1]] * ( sequence_length - len(spans) ) 

        # 必要に応じてtorch.Tensorにする。
        if return_tensors == 'pt':
            encoding = { k: torch.tensor([v]) for k, v in encoding.items() }

        return encoding, spans

    def convert_bert_output_to_entities(self, text, labels, spans):
        """
        文章、ラベル列の予測値、各トークンの位置から固有表現を得る。
        """
        # labels, spansから特殊トークンに対応する部分を取り除く
        labels = [label for label, span in zip(labels, spans) if span[0] != -1]
        spans = [span for span in spans if span[0] != -1]

        # 同じラベルが連続するトークンをまとめて、固有表現を抽出する。
        entities = []
        for label, group \
            in itertools.groupby(enumerate(labels), key=lambda x: x[1]):
            
            group = list(group)
            start = spans[group[0][0]][0]
            end = spans[group[-1][0]][1]

            if label != 0: # ラベルが0以外ならば、新たな固有表現として追加。
                entity = {
                    "name": text[start:end],
                    "span": [start, end],
                    "type_id": label
                }
                entities.append(entity)

        return entities

In [5]:
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.


In [7]:
!git clone --branch v2.0 https://github.com/stockmarkteam/ner-wikipedia-dataset 

Cloning into 'ner-wikipedia-dataset'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 32 (delta 9), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (32/32), done.
Note: checking out 'f7ed83626d90e5a79f1af99775e4b8c6cba15295'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



In [10]:
# データのロード
dataset = json.load(open('ner-wikipedia-dataset/ner.json','r'))

# 固有表現のタイプとIDを対応付る辞書 
type_id_dict = {
    "人名": 1,
    "法人名": 2,
    "政治的組織名": 3,
    "その他の組織名": 4,
    "地名": 5,
    "施設名": 6,
    "製品名": 7,
    "イベント名": 8
}

# カテゴリーをラベルに変更、文字列の正規化する。
for sample in dataset:
    sample['text'] = unicodedata.normalize('NFKC', sample['text'])
    for e in sample["entities"]:
        e['type_id'] = type_id_dict[e['type']]
        del e['type']

# データセットの分割
random.shuffle(dataset)
n = len(dataset)
n_train = int(n*0.6)
n_val = int(n*0.2)
dataset_train = dataset[:n_train]
dataset_val = dataset[n_train:n_train+n_val]
dataset_test = dataset[n_train+n_val:]

print(f"Length of train: {len(dataset_train)}")
print(f"Length of val: {len(dataset_val)}")
print(f"Length of test: {len(dataset_test)}")

Length of train: 3205
Length of val: 1068
Length of test: 1070


In [11]:
def create_dataset(tokenizer, dataset, max_length):
    """
    データセットをデータローダに入力できる形に整形。
    """
    dataset_for_loader = []
    for sample in dataset:
        text = sample['text']
        entities = sample['entities']
        encoding = tokenizer.encode_plus_tagged(
            text, entities, max_length=max_length
        )
        encoding = { k: torch.tensor(v) for k, v in encoding.items() }
        dataset_for_loader.append(encoding)
    return dataset_for_loader

# トークナイザのロード
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

# データセットの作成
max_length = 128
dataset_train_for_loader = create_dataset(
    tokenizer, dataset_train, max_length
)
dataset_val_for_loader = create_dataset(
    tokenizer, dataset_val, max_length
)

# データローダの作成
dataloader_train = DataLoader(
    dataset_train_for_loader, batch_size=32, shuffle=True
)
dataloader_val = DataLoader(dataset_val_for_loader, batch_size=256)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.


In [12]:
next(iter(dataloader_train))

{'input_ids': tensor([[    2,   882,    19,  ...,     0,     0,     0],
         [    2, 19192, 28502,  ...,     0,     0,     0],
         [    2,  2556, 17234,  ...,     0,     0,     0],
         ...,
         [    2,  8417,   853,  ...,     0,     0,     0],
         [    2,   373,  1655,  ...,     0,     0,     0],
         [    2,  2702,  1928,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 2, 2,  ..., 0, 0, 0],
         [0, 5, 0,  ..., 0, 0, 0],
         ...,


In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = BertForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=9
)
# モデルをGPUへ転送
model.to(device)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the m

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [14]:
# 検証
inputs = next(iter(dataloader_train))
#unsqueeze(dim)：元のテンソルを書き換えずに、次元を増やしたテンソルを返す
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)
token_type_ids = inputs["token_type_ids"].to(device)
labels = inputs["labels"].to(device)
print(input_ids.size())
print(attention_mask.size())
print(labels.size())

outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
initial_loss = outputs[0]
initial_loss

torch.Size([32, 128])
torch.Size([32, 128])
torch.Size([32, 128])


tensor(2.5167, device='cuda:0', grad_fn=<NllLossBackward0>)

In [15]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=2e-5)

In [16]:
# 訓練パートの定義
def train(model, epoch):
    model.train() # 訓練モードで実行
    train_loss = 0
    iteration = 0
    for batch in dataloader_train:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch["input_ids"].to(device)
        b_input_mask = batch["attention_mask"].to(device)
        b_labels = batch["labels"].to(device)
        # print(f"size of input_ids: {b_input_ids.size()}")
        # print(f"size of input_mask: {b_input_mask.size()}")
        # print(f"size of labels: {b_labels.size()}")
        # optimizer.zero_grad()
        loss, logits = model(input_ids=b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels,
                        return_dict=False)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()

        if iteration%10 == 0:
          print(f"epoch{epoch}_iter{iteration}: loss_train: {loss}")
        iteration += 1

    return train_loss

# テストパートの定義
def validation(model, epoch):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    iteration = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in dataloader_val:
            b_input_ids = batch["input_ids"].to(device)
            b_input_mask = batch["attention_mask"].to(device)
            b_labels = batch["labels"].to(device)
            with torch.no_grad():        
                loss, logits = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels,
                                    return_dict=False)
            val_loss += loss.item()
            if iteration%10 == 0:
              print(f"epoch{epoch}_iter{iteration}: loss_val: {loss}")
            iteration += 1
    return val_loss

In [17]:
# 学習の実行
max_epoch = 3
train_loss_ = []
test_loss_ = []

for epoch in range(max_epoch):
    train_ = train(model, epoch)
    test_ = validation(model, epoch)
    train_loss_.append(train_)
    test_loss_.append(test_)

# モデル保存
torch.save(model.state_dict(), './model.pth')

epoch0_iter0: loss_train: 2.489518404006958
epoch0_iter10: loss_train: 0.38016951084136963
epoch0_iter20: loss_train: 0.2564806044101715
epoch0_iter30: loss_train: 0.1586759388446808
epoch0_iter40: loss_train: 0.13905303180217743
epoch0_iter50: loss_train: 0.10053282231092453
epoch0_iter60: loss_train: 0.05105551704764366
epoch0_iter70: loss_train: 0.06249164044857025
epoch0_iter80: loss_train: 0.06700285524129868
epoch0_iter90: loss_train: 0.03458094969391823
epoch0_iter100: loss_train: 0.05069100856781006
epoch0_iter0: loss_val: 0.053771596401929855
epoch1_iter0: loss_train: 0.06047069653868675
epoch1_iter10: loss_train: 0.037044476717710495
epoch1_iter20: loss_train: 0.03390757739543915
epoch1_iter30: loss_train: 0.02638479322195053
epoch1_iter40: loss_train: 0.028745219111442566
epoch1_iter50: loss_train: 0.021288486197590828
epoch1_iter60: loss_train: 0.04769686609506607
epoch1_iter70: loss_train: 0.041197240352630615
epoch1_iter80: loss_train: 0.024210793897509575
epoch1_iter90: 

In [18]:
def predict(text, tokenizer, bert_tc):
    """
    BERTで固有表現抽出を行うための関数。
    """
    # 符号化
    encoding, spans = tokenizer.encode_plus_untagged(
        text, return_tensors='pt'
    )
    encoding = { k: v.cuda() for k, v in encoding.items() }

    # ラベルの予測値の計算
    with torch.no_grad():
        output = bert_tc(**encoding)
        scores = output.logits
        labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist() 

    # ラベル列を固有表現に変換
    entities = tokenizer.convert_bert_output_to_entities(
        text, labels_predicted, spans
    )

    return entities

# トークナイザのロード
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

# ファインチューニングしたモデルをロードし、GPUにのせる。
# model = BertForTokenClassification().load_from_checkpoint(
#     "./model.pth"
# )
# bert_tc = model.bert_tc.cuda()


# 固有表現抽出
# 注：以下ではコードのわかりやすさのために、1データづつ処理しているが、
# バッチ化して処理を行った方が処理時間は短い
entities_list = [] # 正解の固有表現を追加していく。
entities_predicted_list = [] # 抽出された固有表現を追加していく。
for sample in tqdm(dataset_test):
    text = sample['text']
    entities_predicted = predict(text, tokenizer, model) # BERTで予測
    entities_list.append(sample['entities'])
    entities_predicted_list.append( entities_predicted )

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.
100%|██████████| 1070/1070 [00:21<00:00, 49.34it/s]


In [23]:
i = 1
print("# 正解 #")
print(entities_list[i])
print("# 推論 #")
print(entities_predicted_list[i])
print("# もとの文章 #")
print(dataset_test[i]["text"])

# 正解 #
[{'name': 'ジョナタン・レイス', 'span': [0, 9], 'type_id': 1}, {'name': 'オラ・トイヴォネン', 'span': [14, 23], 'type_id': 1}, {'name': 'イェレマイン・レンス', 'span': [28, 38], 'type_id': 1}, {'name': 'ジュジャーク・バラージュ', 'span': [43, 55], 'type_id': 1}, {'name': 'オルランド・エンヘラール', 'span': [60, 72], 'type_id': 1}]
# 推論 #
[{'name': 'ジョナタン・レイス', 'span': [0, 9], 'type_id': 1}, {'name': 'オラ・トイヴォネン', 'span': [14, 23], 'type_id': 4}, {'name': 'イェレマイン・レンス', 'span': [28, 38], 'type_id': 4}, {'name': 'ジュジャーク', 'span': [43, 49], 'type_id': 4}, {'name': '・バラ', 'span': [49, 52], 'type_id': 1}, {'name': 'ージュ', 'span': [52, 55], 'type_id': 4}, {'name': 'オルランド・エンヘラール', 'span': [60, 72], 'type_id': 1}]
# もとの文章 #
ジョナタン・レイスが3得点、オラ・トイヴォネンが1得点、イェレマイン・レンスが2得点、ジュジャーク・バラージュが2得点、オルランド・エンヘラールが1得点を挙げた。


In [32]:
def evaluate_model(entities_list, entities_predicted_list, type_id=None):
    """
    正解と予測を比較し、モデルの固有表現抽出の性能を評価する。
    type_idがNoneのときは、全ての固有表現のタイプに対して評価する。
    type_idが整数を指定すると、その固有表現のタイプのIDに対して評価を行う。
    """
    num_entities = 0 # 固有表現(正解)の個数
    num_predictions = 0 # BERTにより予測された固有表現の個数
    num_correct = 0 # BERTにより予測のうち正解であった固有表現の数

    # それぞれの文章で予測と正解を比較。
    # 予測は文章中の位置とタイプIDが一致すれば正解とみなす。
    for entities, entities_predicted in zip(entities_list, entities_predicted_list):

        if type_id:
            entities = [ e for e in entities if e['type_id'] == type_id ]
            entities_predicted = [ 
                e for e in entities_predicted if e['type_id'] == type_id
            ]
            
        get_span_type = lambda e: (e['span'][0], e['span'][1], e['type_id'])
        set_entities = set( get_span_type(e) for e in entities )
        set_entities_predicted = set( get_span_type(e) for e in entities_predicted )

        num_entities += len(entities)
        num_predictions += len(entities_predicted)
        num_correct += len( set_entities & set_entities_predicted )

    # 指標を計算
    precision = num_correct/num_predictions # 適合率
    recall = num_correct/num_entities # 再現率
    f_value = 2*precision*recall/(precision+recall) # F値

    result = {
        'num_entities': num_entities,
        'num_predictions': num_predictions,
        'num_correct': num_correct,
        'precision': precision,
        'recall': recall,
        'f_value': f_value
    }

    return result

In [33]:
# set型の集合演算の検証
set_a = {(8, 11, 2)}
set_b = {(8, 11, 2, 3)}
set_a & set_b

set()

In [41]:
print( evaluate_model(entities_list, entities_predicted_list) )

{'num_entities': 2673, 'num_predictions': 2991, 'num_correct': 2363, 'precision': 0.7900367769976596, 'recall': 0.8840254395809951, 'f_value': 0.8343926553672316}


In [61]:
# 評価結果
import pandas as pd
eval_df = pd.DataFrame()
for k, v in type_id_dict.items():
  eval_res = evaluate_model(entities_list, entities_predicted_list, type_id=v)
  eval_df[k] = eval_res.values()

eval_res_all = evaluate_model(entities_list, entities_predicted_list, type_id=None)
eval_df["ALL"] = eval_res_all.values()

eval_df.index = eval_res_all.keys()
eval_df

,人名,法人名,政治的組織名,その他の組織名,地名,施設名,製品名,イベント名,ALL
num_entities,625.000000,508.000000,236.000000,202.000000,398.000000,260.000000,241.000000,203.000000,2673.000000
num_predictions,608.000000,562.000000,317.000000,260.000000,425.000000,300.000000,257.000000,262.000000,2991.000000
num_correct,577.000000,454.000000,210.000000,175.000000,336.000000,234.000000,189.000000,188.000000,2363.000000
precision,0.949013,0.807829,0.662461,0.673077,0.790588,0.780000,0.735409,0.717557,0.790037
recall,0.923200,0.893701,0.889831,0.866337,0.844221,0.900000,0.784232,0.926108,0.884025
f_value,0.935929,0.848598,0.759494,0.757576,0.816525,0.835714,0.759036,0.808602,0.834393


In [53]:
type_id_dict.keys()

dict_keys(['人名', '法人名', '政治的組織名', 'その他の組織名', '地名', '施設名', '製品名', 'イベント名'])

In [57]:
eval_res.keys()

dict_keys(['num_entities', 'num_predictions', 'num_correct', 'precision', 'recall', 'f_value'])